In [1]:
import os
import re
import glob
import yaml
import argparse

import numpy as np
import pandas as pd
from matplotlib import cm

from math import ceil

import zarr
import napari
import tifffile
import dask.array as da

from utils.utility_functions import single_channel_pyramid

In [2]:
# connect to "smb://files.med.harvard.edu/ImStor/sorger/data/RareCyte" to read SARDANA-102 image data

In [3]:
# I/O

# read OME-TIFF and segmentation outlines
tif_path = '/Volumes/RareCyte/JL503_JERRY/TNP_2020/WD-76845-102.ome.tif'
seg_path = (
    '/Volumes/RareCyte/JL503_JERRY/TNP_2020/mcmicro-20200922/'
    'WD-76845-102/qc/s3seg/unmicst-WD-76845-102/nucleiRingOutlines.tif'
)

# read unclustered single-cell data or SARDANA-102 image 
sc_data_path = '/Volumes/RareCyte/JL503_JERRY/TNP_2020/mcmicro-20200922/WD-76845-102.csv'
data = pd.read_csv(sc_data_path)

# import markers.csv
markers = pd.read_csv(os.path.join(os.getcwd(), 'input/CRC-097_mcmicro_markers.csv'))

# get markers excluded from analysis
with open(os.path.join(os.getcwd(), 'input/CRC-97_cylinter_config.yml')) as f:
    config = yaml.safe_load(f)
markers_to_exclude = config['markersToExclude']


# import image contrast settings
with open(os.path.join(os.getcwd(), 'input/CRC-097_cylinter_contrast_limits.yml')) as f:
    contrast_limits = yaml.safe_load(f)

# The parquet file at the path below is being read because "main.csv" 
# uses trimmed marker channel names as column headers that differ from the raw channel names used 
# in the markers.csv file used to index channels in the OME-TIFF image.
for_channels = pd.read_parquet(
    os.path.join(os.getcwd(), 'input/CRC-097_clean_cylinter_clustering_3d_leiden.parquet')
)

# isolate antibodies of interest
abx_channels = [
    i for i in for_channels.columns if 'nucleiRingMask' in i if
    'Hoechst' not in i if i not in markers_to_exclude
]

# get name of first DNA channel
dna1 = markers['marker_name'][markers['channel_number'] == 1][0]
dna_moniker = str(re.search(r'[^\W\d]+', dna1).group())

# read cluster labels for SARADANA-102 (VAE20 model applied, HDBSCAN clustering)
clusters = pd.read_csv(os.path.join(os.getcwd(), 'input/vae102_clustering_solution_MCS150_NN30_RS3_Rand3.csv'))

# merge with single-cell dataframe
data = data.merge(clusters, how='inner', on='CellID')

In [4]:
# initialize image viewer
viewer = napari.Viewer()

--Return--
None
> /var/folders/_h/pbzrx8ss6n5f031pf4hc97_w0000gp/T/ipykernel_37909/1141651035.py(2)<module>()
      1 # initialize image viewer
----> 2 import pdb; pdb.set_trace()
      3 viewer = napari.Viewer()



ipdb>  data.columns


Index(['CellID', 'Hoechst0_cellRingMask', 'AF488_cellRingMask',
       'AF555_cellRingMask', 'AF647_cellRingMask', 'Hoechst1_cellRingMask',
       'A488_cellRingMask', 'A555_cellRingMask', 'A647_cellRingMask',
       'Hoechst2_cellRingMask',
       ...
       'CDX2_647_nucleiRingMask', 'Hoechst8_nucleiRingMask',
       'LaminABC_488_nucleiRingMask', 'Desmin_555_nucleiRingMask',
       'CD31_647_nucleiRingMask', 'Hoechst9_nucleiRingMask',
       'PCNA_488_nucleiRingMask', 'Ki67_570_nucleiRingMask',
       'CollagenIV_647_nucleiRingMask', 'vae_cluster'],
      dtype='object', length=133)


ipdb>  c


In [5]:
# add DNA1 channel to Napari image viewer
dna, min, max = single_channel_pyramid(glob.glob(tif_path)[0], channel=0)
viewer.add_image(
    dna, rgb=False, blending='additive',
    colormap='gray', visible=True, opacity=1.0,
    name='DNA1', contrast_limits=(min, max)
)

<Image layer 'DNA1' at 0x15934ec40>

In [6]:
# add marker channels and apply previously defined contrast limits
for ch in abx_channels:
    ch = ch.rsplit('_', 1)[0]
    channel_number = markers['channel_number'][markers['marker_name'] == ch]
    img, min, max = single_channel_pyramid(
        glob.glob(tif_path)[0], channel=(channel_number.item() - 1)
    )
    viewer.add_image(
        img, rgb=False, blending='additive', colormap='green', visible=False, name=ch,
        contrast_limits=(min, max)
    )
for ch in abx_channels:
    ch = ch.rsplit('_', 1)[0]
    viewer.layers[ch].contrast_limits = (
        contrast_limits[ch][0], contrast_limits[ch][1])

OSError: [Errno 6] Device not configured

In [ ]:
# add centroids of cells for each VAE cluster to image viewer
num_colors = len(list(cm.tab20.colors))
num_clusters = len(data['vae_cluster'].unique())
palette_multiplier = ceil(num_clusters / num_colors)
colors = list(cm.tab20.colors) * palette_multiplier
colors = colors[0:num_clusters]
colors.reverse()

for c, cluster in zip(colors, sorted(data['vae_cluster'].unique(), reverse=True)):
    centroids = data[['Y_centroid', 'X_centroid']][data['vae_cluster'] == cluster]
    viewer.add_points(
        centroids, name=f'VAE{cluster}', face_color=np.array(c), edge_color='white',
        edge_width=0.0, size=3.0, opacity=1.0, blending='translucent', visible=False
    )

In [ ]:
# read segmentation outlines
seg, min, max = single_channel_pyramid(glob.glob(seg_path)[0], channel=0)
viewer.add_image(
    seg, rgb=False, blending='additive',
    colormap='gray', visible=False,
    name='segmentation', opacity=0.3, contrast_limits=(min, max)
)

In [ ]:
# run image viewer
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'um'

napari.run()